## Fase 2

In [1]:
# ============================================================================
# FASE 2 - OPTIMIZADO PARA SPARK 3.5.1 + DELTA LAKE 3.0
# ============================================================================

# PASO 0: REINICIAR SPARK CON VERSIONES CORRECTAS
try:
    spark.stop()
except:
    pass

import time
time.sleep(3)

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import col, lower, regexp_replace, translate, length, trim

from pyspark.ml.feature import (
    Tokenizer, StopWordsRemover, Word2Vec, 
    StringIndexer, OneHotEncoder, VectorAssembler,
    StandardScaler, PCA
)
from pyspark.ml import Pipeline
from pyspark.ml.stat import Correlation
import numpy as np

spark = (
    SparkSession.builder
    .appName("Bronze_to_Silver_Optimized")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.jars.packages", 
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1,"
            "io.delta:delta-spark_2.12:3.0.0")
    .config("spark.driver.memory", "4g")
    .config("spark.executor.memory", "4g")
    .config("spark.driver.maxResultSize", "2g")
    .config("spark.sql.shuffle.partitions", "50")
    .config("spark.sql.adaptive.enabled", "true")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")
print(f" Spark {spark.version} iniciado\n")


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bda52606-2bf2-4a26-a4d2-aef53be004ae;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
	found io.delta#delta-spark_

 Spark 3.5.1 iniciado



In [3]:
# ============================================================================
# 1. LECTURA DE KAFKA
# ============================================================================

print("="*80)
print("PASO 1: LECTURA DE KAFKA")
print("="*80 + "\n")

contract_schema = StructType([
    StructField("id_contrato", StringType()),
    StructField("objeto_contrato", StringType()),
    StructField("entidad", StringType()),
    StructField("departamento", StringType()),
    StructField("municipio", StringType()),
    StructField("region", StringType()),
    StructField("codigo_unspsc", StringType()),
    StructField("descripcion_categoria", StringType()),
    StructField("valor_contrato", DoubleType()),
    StructField("duracion_dias", IntegerType()),
    StructField("fecha_firma", StringType()),
    StructField("tipo_contrato", StringType()),
    StructField("estado_contrato", StringType()),
    StructField("modalidad", StringType()),
    StructField("anno", IntegerType()),
    StructField("id_interno_sistema", StringType()),
    StructField("campo_vacio", StringType()),
    StructField("constante_1", StringType()),
    StructField("constante_2", IntegerType()),
    StructField("duplicate_id", StringType()),
    StructField("timestamp_carga", StringType())
])

print("Leyendo Kafka...")

df_kafka = spark.read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "contratos-publicos") \
    .option("startingOffsets", "earliest") \
    .load()

df_bronze = df_kafka.select(
    from_json(col("value").cast("string"), contract_schema).alias("data")
).select("data.*")

df_bronze = df_bronze.cache()
total_kafka = df_bronze.count()

print(f" Mensajes: {total_kafka:,}\n")



PASO 1: LECTURA DE KAFKA

Leyendo Kafka...


[Stage 0:>                                                          (0 + 1) / 1]

 Mensajes: 50,349



In [4]:
# ============================================================================
# 2. ELIMINAR REDUNDANTES
# ============================================================================

print("="*80)
print("PASO 2: ELIMINAR REDUNDANTES")
print("="*80 + "\n")

redundant_columns = [
    "id_interno_sistema", "campo_vacio", "constante_1",
    "constante_2", "duplicate_id", "timestamp_carga"
]

print(f" Eliminando {len(redundant_columns)} columnas redundantes")

df_cleaned = df_bronze.drop(*redundant_columns)

print(f" Columnas restantes: {len(df_cleaned.columns)}\n")

#  LIBERAR bronze, ya no lo necesitamos
df_bronze.unpersist()

PASO 2: ELIMINAR REDUNDANTES

 Eliminando 6 columnas redundantes
 Columnas restantes: 15



DataFrame[id_contrato: string, objeto_contrato: string, entidad: string, departamento: string, municipio: string, region: string, codigo_unspsc: string, descripcion_categoria: string, valor_contrato: double, duracion_dias: int, fecha_firma: string, tipo_contrato: string, estado_contrato: string, modalidad: string, anno: int, id_interno_sistema: string, campo_vacio: string, constante_1: string, constante_2: int, duplicate_id: string, timestamp_carga: string]

In [5]:
# ============================================================================
# 3. LIMPIEZA
# ============================================================================

print("="*80)
print("PASO 3: LIMPIEZA")
print("="*80 + "\n")

df_cleaned = df_cleaned.cache()
total_cleaned = df_cleaned.count()

print(f" Registros: {total_cleaned:,}\n")

# Análisis de nulos optimizado
null_counts = df_cleaned.select([
    count(when(col(c).isNull(), c)).alias(c) 
    for c in df_cleaned.columns
]).collect()[0].asDict()

print("Columnas con nulos:")
for col_name, null_count in sorted(null_counts.items(), key=lambda x: x[1], reverse=True):
    if null_count > 0:
        pct = (null_count / total_cleaned) * 100
        print(f"   {col_name}: {null_count:,} ({pct:.1f}%)")

# Limpieza
df_silver = df_cleaned \
    .filter(col("id_contrato").isNotNull()) \
    .filter(col("objeto_contrato").isNotNull()) \
    .filter(col("valor_contrato").isNotNull()) \
    .filter(col("valor_contrato") > 0) \
    .filter(col("fecha_firma").isNotNull()) \
    .withColumn("fecha_firma", to_date(col("fecha_firma"), "yyyy-MM-dd"))

df_silver = df_silver.cache()
total_silver = df_silver.count()

print(f"\n Limpieza:")
print(f"   Antes: {total_cleaned:,}")
print(f"   Después: {total_silver:,}")
print(f"   Descartados: {total_cleaned - total_silver:,}\n")

df_cleaned.unpersist()


PASO 3: LIMPIEZA



 Registros: 50,349

Columnas con nulos:
   duracion_dias: 50,349 (100.0%)

 Limpieza:
   Antes: 50,349
   Después: 50,058
   Descartados: 291



DataFrame[id_contrato: string, objeto_contrato: string, entidad: string, departamento: string, municipio: string, region: string, codigo_unspsc: string, descripcion_categoria: string, valor_contrato: double, duracion_dias: int, fecha_firma: string, tipo_contrato: string, estado_contrato: string, modalidad: string, anno: int]

In [6]:
# ============================================================================
# 4. ESTADÍSTICAS
# ============================================================================

print("="*80)
print("PASO 4: ESTADÍSTICAS")
print("="*80 + "\n")

print("📊 Por región:")
df_silver.groupBy("region").count().orderBy(desc("count")).show(5)

print("\n📊 Top 5 entidades:")
df_silver.groupBy("entidad").count().orderBy(desc("count")).show(5, truncate=False)


PASO 4: ESTADÍSTICAS

📊 Por región:
+--------------+-----+
|        region|count|
+--------------+-----+
|Centro-Oriente|50058|
+--------------+-----+


📊 Top 5 entidades:
+-------------------------------------------------+-----+
|entidad                                          |count|
+-------------------------------------------------+-----+
|MUNICIPIO DE SOACHA.                             |3184 |
|ALCALDÍA MUNICIPAL COTA                          |1995 |
|ESE MUNICIPAL DE SOACHA JULIO CESAR PEÑALOZA*    |1919 |
|CUNDINAMARCA-ALCALDIA MUNICIPIO MOSQUERA         |1879 |
|empresa social del estado región de salud soacha.|1579 |
+-------------------------------------------------+-----+
only showing top 5 rows



In [7]:
# ============================================================================
# 5. GUARDAR EN DELTA LAKE
# ============================================================================

print("="*80)
print("PASO 5: GUARDAR EN DELTA LAKE")
print("="*80 + "\n")

DELTA_PATH = "/app/notebooks/delta_lake/silver_contracts"

print(f"💾 Guardando en: {DELTA_PATH}")

df_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(DELTA_PATH)

print("✅ Guardado exitosamente\n")

# ⚠️ LIBERAR todo
df_silver.unpersist()
spark.catalog.clearCache()


PASO 5: GUARDAR EN DELTA LAKE

💾 Guardando en: /app/notebooks/delta_lake/silver_contracts


✅ Guardado exitosamente



In [8]:
print(f"Spark version: {spark.version}")

Spark version: 3.5.1


In [9]:
# ============================================================================
# 6. VERIFICACIÓN
# ============================================================================

print("="*80)
print("VERIFICACIÓN FINAL")
print("="*80 + "\n")

df_verify = spark.read.format("delta").load(DELTA_PATH)
print(f"✅ Registros verificados: {df_verify.count():,}")

df_verify.select("id_contrato", "entidad", "valor_contrato", "fecha_firma") \
    .show(5, truncate=False)

print("\n🎯 Fase 2 completada. Siguiente: Fase 3 - Embeddings\n")

VERIFICACIÓN FINAL

✅ Registros verificados: 50,058
+------------+----------------------------------------------------+--------------+-----------+
|id_contrato |entidad                                             |valor_contrato|fecha_firma|
+------------+----------------------------------------------------+--------------+-----------+
|011-2024    |ESE HOSPITAL SALAZAR DE VILLETA                     |2.9991E7      |2024-01-01 |
|CPS 012-2024|E.S.E HOSPITAL NUESTRA SEÑORA DEL CARMEN DEL COLEGIO|1.07844E7     |2024-01-01 |
|CPS 017-2024|E.S.E HOSPITAL NUESTRA SEÑORA DEL CARMEN DEL COLEGIO|1.07844E7     |2024-01-01 |
|004-2024    |ESE HOSPITAL SALAZAR DE VILLETA                     |2.016E7       |2024-01-01 |
|CPS-042-2024|empresa social del estado región de salud soacha.   |7.5323616E7   |2024-01-01 |
+------------+----------------------------------------------------+--------------+-----------+
only showing top 5 rows


🎯 Fase 2 completada. Siguiente: Fase 3 - Embeddings



## Fase 3

In [10]:
# ============================================================================
# 1. CARGAR DATOS DESDE SILVER
# ============================================================================

print("\n" + "="*80)
print("PASO 1: CARGAR DATOS DESDE SILVER")
print("="*80 + "\n")

SILVER_PATH = "/app/notebooks/delta_lake/silver_contracts"

print(f"📖 Cargando datos desde: {SILVER_PATH}")

df_silver = spark.read.format("delta").load(SILVER_PATH)
df_silver = df_silver.cache()

total_records = df_silver.count()
print(f"✅ Registros cargados: {total_records:,}\n")

print("📋 Esquema de datos:")
df_silver.printSchema()

print("\n📊 Muestra de datos:")
df_silver.select(
    "id_contrato", "objeto_contrato", "entidad", 
    "codigo_unspsc", "valor_contrato", "duracion_dias"
).show(3, truncate=False)





PASO 1: CARGAR DATOS DESDE SILVER

📖 Cargando datos desde: /app/notebooks/delta_lake/silver_contracts
✅ Registros cargados: 50,058

📋 Esquema de datos:
root
 |-- id_contrato: string (nullable = true)
 |-- objeto_contrato: string (nullable = true)
 |-- entidad: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- region: string (nullable = true)
 |-- codigo_unspsc: string (nullable = true)
 |-- descripcion_categoria: string (nullable = true)
 |-- valor_contrato: double (nullable = true)
 |-- duracion_dias: integer (nullable = true)
 |-- fecha_firma: date (nullable = true)
 |-- tipo_contrato: string (nullable = true)
 |-- estado_contrato: string (nullable = true)
 |-- modalidad: string (nullable = true)
 |-- anno: integer (nullable = true)


📊 Muestra de datos:
+------------+---------------------------------------------------------------------------------------------------------------------------------------+-----------------

In [11]:
# ============================================================================
# 2. LIMPIEZA Y PREPARACIÓN DE TEXTO
# ============================================================================

print("\n" + "="*80)
print("PASO 2: LIMPIEZA Y PREPARACIÓN DE TEXTO")
print("="*80 + "\n")

print("🧹 Limpiando columna 'objeto_contrato'...")

# Limpiar y normalizar texto
df_prepared = df_silver.withColumn(
    "objeto_limpio",
    lower(
        regexp_replace(
            regexp_replace(col("objeto_contrato"), "[^a-zA-ZáéíóúñÑ0-9\\s]", " "),
            "\\s+", " "
        )
    )
)

# Filtrar textos muy cortos (menos de 10 caracteres)
df_prepared = df_prepared.filter(length(col("objeto_limpio")) >= 10)

print(f"✅ Registros después de limpieza: {df_prepared.count():,}")

print("\n📋 Ejemplo de texto limpio:")
df_prepared.select("objeto_contrato", "objeto_limpio").show(2, truncate=80)



PASO 2: LIMPIEZA Y PREPARACIÓN DE TEXTO

🧹 Limpiando columna 'objeto_contrato'...


✅ Registros después de limpieza: 50,058

📋 Ejemplo de texto limpio:
+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|                                                                 objeto_contrato|                                                                   objeto_limpio|
+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|PRESTACION DE SERVICIOS ASISTENCIALES COMO MEDICO GENERAL EN LOS PROCESOS Y S...|prestacion de servicios asistenciales como medico general en los procesos y s...|
|PRESTAR APOYO AL PROCESO ASISTENCIAL EN EL ÁREA DE AUXILIAR DE ENFERMERÍA A L...|prestar apoyo al proceso asistencial en el rea de auxiliar de enfermer a a la...|
+--------------------------------------------------------------------------------+------------------------------

In [12]:
# ============================================================================
# 3. TOKENIZACIÓN Y STOPWORDS
# ============================================================================

print("\n" + "="*80)
print("PASO 3: TOKENIZACIÓN Y REMOCIÓN DE STOPWORDS")
print("="*80 + "\n")

# Tokenización
tokenizer = Tokenizer(inputCol="objeto_limpio", outputCol="palabras")
df_tokenized = tokenizer.transform(df_prepared)

print("✅ Tokenización completada")
print("\n📋 Ejemplo de tokens:")
df_tokenized.select("objeto_limpio", "palabras").show(2, truncate=80)

# Stopwords en español
stopwords_es = [
    "el", "la", "de", "que", "y", "a", "en", "un", "ser", "se", "no", "haber",
    "por", "con", "su", "para", "como", "estar", "tener", "le", "lo", "todo",
    "pero", "más", "hacer", "o", "poder", "decir", "este", "ir", "otro", "ese",
    "si", "me", "ya", "ver", "porque", "dar", "cuando", "él", "muy", "sin",
    "vez", "mucho", "saber", "qué", "sobre", "mi", "alguno", "mismo", "yo",
    "también", "hasta", "año", "dos", "querer", "entre", "así", "primero",
    "desde", "grande", "eso", "ni", "nos", "llegar", "pasar", "tiempo", "ella",
    "del", "al", "los", "las", "uno", "una", "unos", "unas"
]

# Remover stopwords
remover = StopWordsRemover(
    inputCol="palabras", 
    outputCol="palabras_filtradas",
    stopWords=stopwords_es
)
df_filtered = remover.transform(df_tokenized)

print("✅ Stopwords removidas")
print("\n📋 Comparación antes/después de stopwords:")
df_filtered.select("palabras", "palabras_filtradas").show(2, truncate=80)

# Filtrar documentos sin palabras después de stopwords
df_filtered = df_filtered.filter(size(col("palabras_filtradas")) > 0)

print(f"\n📊 Registros con palabras válidas: {df_filtered.count():,}")




PASO 3: TOKENIZACIÓN Y REMOCIÓN DE STOPWORDS

✅ Tokenización completada

📋 Ejemplo de tokens:
+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|                                                                   objeto_limpio|                                                                        palabras|
+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|prestacion de servicios asistenciales como medico general en los procesos y s...|[prestacion, de, servicios, asistenciales, como, medico, general, en, los, pr...|
|prestar apoyo al proceso asistencial en el rea de auxiliar de enfermer a a la...|[prestar, apoyo, al, proceso, asistencial, en, el, rea, de, auxiliar, de, enf...|
+--------------------------------------------------------------------------------+---

[Stage 50:=============================>                            (1 + 1) / 2]


📊 Registros con palabras válidas: 50,058
